In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in /Users/jlh/opt/anaconda3/lib/python3.8/site-packages (0.0)


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [34]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
import os
import numpy as np

# Read the CSV and Perform Basic Data Cleaning

In [35]:
df = pd.read_csv("cumulative.csv")
# Drop the null columns where all values are null
df = df.drop(columns=["kepler_name"])
df.head()

,rowid,kepid,kepoi_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,1,10797460,K00752.01,CONFIRMED,CANDIDATE,1.000,0,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,2,10797460,K00752.02,CONFIRMED,CANDIDATE,0.969,0,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,3,10811496,K00753.01,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,0,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,4,10848459,K00754.01,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,0,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,5,10854555,K00755.01,CONFIRMED,CANDIDATE,1.000,0,0,0,0,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [36]:
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,rowid,kepid,kepoi_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,1,10797460,K00752.01,CONFIRMED,CANDIDATE,1.000,0,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,2,10797460,K00752.02,CONFIRMED,CANDIDATE,0.969,0,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,3,10811496,K00753.01,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,0,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,4,10848459,K00754.01,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,0,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,5,10854555,K00755.01,CONFIRMED,CANDIDATE,1.000,0,0,0,0,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [37]:
df = df.drop(columns=["rowid", "kepid", "koi_pdisposition", "koi_score", "koi_tce_delivname"])
df.head()

,kepoi_name,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,K00752.01,CONFIRMED,0,0,0,0,9.488036,2.775000e-05,-2.775000e-05,170.538750,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,K00752.02,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,K00753.01,FALSE POSITIVE,0,1,0,0,19.899140,1.494000e-05,-1.494000e-05,175.850252,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,K00754.01,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,K00755.01,CONFIRMED,0,0,0,0,2.525592,3.761000e-06,-3.761000e-06,171.595550,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


# Select your features (columns)

In [38]:
# Set features. This will also be used as your x values.
#selected_features = df[['koi_disposition', 'koi_period', 'koi_time0bk', 'koi_impact', 'koi_duration', 'koi_depth', 'koi_prad', 'koi_teq', 'koi_insol', 'koi_model_snr', 'koi_steff', 'koi_slogg', 'koi_srad']]
#selected_features
X = df.drop(columns=["koi_disposition", "kepoi_name"], axis=1)
print(X.shape)

(7803, 40)


# Create a Train Test Split

Use `koi_disposition` for the y values

In [39]:
y = df["koi_disposition"]
print(y.shape)

(7803,)


In [40]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [41]:
X_train.head()
y_train.head()

7556         CANDIDATE
2272         CANDIDATE
5020         CONFIRMED
7434    FALSE POSITIVE
5058         CANDIDATE
Name: koi_disposition, dtype: object

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [42]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [43]:
from tensorflow.keras.utils import to_categorical

# Step 2: One-hot encoding
hot_y_train = to_categorical(encoded_y_train)
hot_y_test = to_categorical(encoded_y_test)
hot_y_train

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.]], dtype=float32)

In [44]:
hot_y_train.shape

(5852, 3)

In [45]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [46]:
X_train_scaled.shape

(5852, 40)

# Train the Model



In [47]:
# Support vector machine linear classifier
from tensorflow.keras.models import Sequential

model = Sequential()

In [48]:
# first, create a normal neural network with 2 inputs, 6 hidden nodes, and 2 outputs
from tensorflow.keras.layers import Dense
number_inputs = 40
number_hidden_nodes = 6
model.add(Dense(units=number_hidden_nodes,
                activation='relu', input_dim=number_inputs))

In [49]:
number_classes = 3
model.add(Dense(units=number_classes, activation='softmax'))

In [50]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 6)                 246       
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 21        
Total params: 267
Trainable params: 267
Non-trainable params: 0
_________________________________________________________________


In [51]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [52]:
# Fit the model to the training data
model.fit(
    X_train_scaled,
    hot_y_train,
    epochs=150,
    shuffle=True,
    verbose=2
)

Epoch 1/150
183/183 - 0s - loss: 1.0623 - accuracy: 0.4074
Epoch 2/150
183/183 - 0s - loss: 0.9505 - accuracy: 0.4836
Epoch 3/150
183/183 - 0s - loss: 0.7538 - accuracy: 0.5439
Epoch 4/150
183/183 - 0s - loss: 0.6302 - accuracy: 0.7582
Epoch 5/150
183/183 - 0s - loss: 0.5656 - accuracy: 0.7659
Epoch 6/150
183/183 - 0s - loss: 0.5254 - accuracy: 0.7662
Epoch 7/150
183/183 - 0s - loss: 0.4980 - accuracy: 0.7667
Epoch 8/150
183/183 - 0s - loss: 0.4785 - accuracy: 0.7666
Epoch 9/150
183/183 - 0s - loss: 0.4643 - accuracy: 0.7671
Epoch 10/150
183/183 - 0s - loss: 0.4533 - accuracy: 0.7679
Epoch 11/150
183/183 - 0s - loss: 0.4447 - accuracy: 0.7686
Epoch 12/150
183/183 - 0s - loss: 0.4380 - accuracy: 0.7693
Epoch 13/150
183/183 - 0s - loss: 0.4323 - accuracy: 0.7703
Epoch 14/150
183/183 - 0s - loss: 0.4273 - accuracy: 0.7719
Epoch 15/150
183/183 - 0s - loss: 0.4234 - accuracy: 0.7734
Epoch 16/150
183/183 - 0s - loss: 0.4194 - accuracy: 0.7753
Epoch 17/150
183/183 - 0s - loss: 0.4148 - accura

183/183 - 0s - loss: 0.3025 - accuracy: 0.8662
Epoch 138/150
183/183 - 0s - loss: 0.3017 - accuracy: 0.8669
Epoch 139/150
183/183 - 0s - loss: 0.3018 - accuracy: 0.8672
Epoch 140/150
183/183 - 0s - loss: 0.3004 - accuracy: 0.8689
Epoch 141/150
183/183 - 0s - loss: 0.3022 - accuracy: 0.8669
Epoch 142/150
183/183 - 0s - loss: 0.3011 - accuracy: 0.8696
Epoch 143/150
183/183 - 0s - loss: 0.3018 - accuracy: 0.8648
Epoch 144/150
183/183 - 0s - loss: 0.2998 - accuracy: 0.8713
Epoch 145/150
183/183 - 0s - loss: 0.3014 - accuracy: 0.8700
Epoch 146/150
183/183 - 0s - loss: 0.3001 - accuracy: 0.8669
Epoch 147/150
183/183 - 0s - loss: 0.3008 - accuracy: 0.8650
Epoch 148/150
183/183 - 0s - loss: 0.3033 - accuracy: 0.8650
Epoch 149/150
183/183 - 0s - loss: 0.3007 - accuracy: 0.8679
Epoch 150/150
183/183 - 0s - loss: 0.2995 - accuracy: 0.8674


In [53]:
# Evaluate the model using the testing data
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, hot_y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

61/61 - 0s - loss: 0.3137 - accuracy: 0.8770
Loss: 0.3136589229106903, Accuracy: 0.8769861459732056


In [72]:
#Continue with Deep Learning
from tensorflow.keras.models import Sequential

deepmodel = Sequential()

In [73]:
from tensorflow.keras.layers import Dense
number_inputs = 40
number_hidden_nodes = 8
deepmodel.add(Dense(units=number_hidden_nodes,
                activation='relu', input_dim=number_inputs))
deepmodel.add(Dense(units=6, activation='softmax'))
deepmodel.add(Dense(units=4, activation='relu'))

In [74]:
number_classes = 3
deepmodel.add(Dense(units=number_classes, activation='softmax'))

In [75]:
deepmodel.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 8)                 328       
_________________________________________________________________
dense_13 (Dense)             (None, 6)                 54        
_________________________________________________________________
dense_14 (Dense)             (None, 4)                 28        
_________________________________________________________________
dense_15 (Dense)             (None, 3)                 15        
Total params: 425
Trainable params: 425
Non-trainable params: 0
_________________________________________________________________


In [76]:
# Compile the model
deepmodel.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [77]:
hot_y_train.shape

(5852, 3)

In [78]:
X_train_scaled.shape

(5852, 40)

In [79]:
# Fit the model to the training data
deepmodel.fit(
    X_train_scaled,
    hot_y_train,
    epochs=200,
    shuffle=True,
    verbose=2
)

Epoch 1/200
183/183 - 0s - loss: 1.0408 - accuracy: 0.5044
Epoch 2/200
183/183 - 0s - loss: 0.7769 - accuracy: 0.7539
Epoch 3/200
183/183 - 0s - loss: 0.5439 - accuracy: 0.7664
Epoch 4/200
183/183 - 0s - loss: 0.4597 - accuracy: 0.7671
Epoch 5/200
183/183 - 0s - loss: 0.4302 - accuracy: 0.7673
Epoch 6/200
183/183 - 0s - loss: 0.4171 - accuracy: 0.7674
Epoch 7/200
183/183 - 0s - loss: 0.4098 - accuracy: 0.7669
Epoch 8/200
183/183 - 0s - loss: 0.4064 - accuracy: 0.7673
Epoch 9/200
183/183 - 0s - loss: 0.4033 - accuracy: 0.7679
Epoch 10/200
183/183 - 0s - loss: 0.4011 - accuracy: 0.7673
Epoch 11/200
183/183 - 0s - loss: 0.4016 - accuracy: 0.7667
Epoch 12/200
183/183 - 0s - loss: 0.3998 - accuracy: 0.7685
Epoch 13/200
183/183 - 0s - loss: 0.3985 - accuracy: 0.7673
Epoch 14/200
183/183 - 0s - loss: 0.3984 - accuracy: 0.7673
Epoch 15/200
183/183 - 0s - loss: 0.3972 - accuracy: 0.7681
Epoch 16/200
183/183 - 0s - loss: 0.3975 - accuracy: 0.7667
Epoch 17/200
183/183 - 0s - loss: 0.3962 - accura

183/183 - 0s - loss: 0.3250 - accuracy: 0.8582
Epoch 138/200
183/183 - 0s - loss: 0.3228 - accuracy: 0.8600
Epoch 139/200
183/183 - 0s - loss: 0.3217 - accuracy: 0.8595
Epoch 140/200
183/183 - 0s - loss: 0.3240 - accuracy: 0.8630
Epoch 141/200
183/183 - 0s - loss: 0.3212 - accuracy: 0.8636
Epoch 142/200
183/183 - 0s - loss: 0.3204 - accuracy: 0.8643
Epoch 143/200
183/183 - 0s - loss: 0.3208 - accuracy: 0.8630
Epoch 144/200
183/183 - 0s - loss: 0.3209 - accuracy: 0.8665
Epoch 145/200
183/183 - 0s - loss: 0.3192 - accuracy: 0.8635
Epoch 146/200
183/183 - 0s - loss: 0.3189 - accuracy: 0.8652
Epoch 147/200
183/183 - 0s - loss: 0.3189 - accuracy: 0.8657
Epoch 148/200
183/183 - 0s - loss: 0.3166 - accuracy: 0.8672
Epoch 149/200
183/183 - 0s - loss: 0.3161 - accuracy: 0.8652
Epoch 150/200
183/183 - 0s - loss: 0.3159 - accuracy: 0.8671
Epoch 151/200
183/183 - 0s - loss: 0.3161 - accuracy: 0.8683
Epoch 152/200
183/183 - 0s - loss: 0.3171 - accuracy: 0.8664
Epoch 153/200
183/183 - 0s - loss: 0.3

In [80]:
# Evaluate the model using the testing data
deepmodel_loss, deepmodel_accuracy = deepmodel.evaluate(
    X_test_scaled, hot_y_test, verbose=2)
print(f"Loss: {deepmodel_loss}, Accuracy: {deepmodel_accuracy}")

61/61 - 0s - loss: 0.2991 - accuracy: 0.8754
Loss: 0.29905611276626587, Accuracy: 0.87544846534729


In [84]:
encoded_predictions = deepmodel.predict_classes(X_test_scaled[:20])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [85]:
print(f"Predicted Classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:20])}")

Predicted Classes: ['CANDIDATE' 'CONFIRMED' 'FALSE POSITIVE' 'FALSE POSITIVE' 'CANDIDATE'
 'FALSE POSITIVE' 'CONFIRMED' 'FALSE POSITIVE' 'FALSE POSITIVE'
 'CANDIDATE' 'CONFIRMED' 'CANDIDATE' 'FALSE POSITIVE' 'FALSE POSITIVE'
 'FALSE POSITIVE' 'FALSE POSITIVE' 'CANDIDATE' 'CONFIRMED' 'CANDIDATE'
 'FALSE POSITIVE']
Actual Labels: ['CANDIDATE', 'CONFIRMED', 'FALSE POSITIVE', 'FALSE POSITIVE', 'CANDIDATE', 'FALSE POSITIVE', 'CONFIRMED', 'FALSE POSITIVE', 'FALSE POSITIVE', 'CANDIDATE', 'CANDIDATE', 'CANDIDATE', 'FALSE POSITIVE', 'FALSE POSITIVE', 'FALSE POSITIVE', 'FALSE POSITIVE', 'CONFIRMED', 'CONFIRMED', 'CANDIDATE', 'FALSE POSITIVE']


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

# Save the Model

In [68]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
#import joblib
#filename = 'your_name.sav'
#joblib.dump(your_model, filename)
model.save("exoplanetNeuralNetwork.h5")
deepmodel.save("exoplanetNeuralNetworkDeep.h5")